In [ ]:
import pandas as pd
import pyodbc

df = pd.read_csv(r'shedulers.csv', encoding='cp1251', sep=';')

In [ ]:
df.columns = ['NAME', 'SHEDULE', 'DATE_BEGIN', 'DATE_END']
df.DATE_BEGIN = pd.to_datetime(df.DATE_BEGIN, errors = 'coerce', format='%d.%m.%Y')
df.DATE_END = pd.to_datetime(df.DATE_END, errors = 'coerce', format='%d.%m.%Y')
end = pd.Timestamp.max
df.DATE_END = df.DATE_END.fillna(end)
df.info()

# Задание 1
``` mysql
    if object_id('dbo.T_CONTRACTOR_SHERULER', 'U') is not null
        drop table DEBT.dbo.T_CONTRACTOR_SHERULER;

    CREATE TABLE DBO.T_CONTRACTOR_SHERULER (
        ID_NAME INT NOT NULL  IDENTITY(1, 1),
        NAME VARCHAR(255),
        SHEDULE VARCHAR(255),
        DATE_BEGIN DATETIME,
        DATE_END DATETIME,
        PRIMARY KEY (ID_NAME),
        CONSTRAINT un_NAME UNIQUE (NAME, DATE_BEGIN),
        CONSTRAINT check_date CHECK (
            DATE_BEGIN < DATE_END
            )
        )
```

In [ ]:
conn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};'
                          'SERVER=DCSQLREP;'
                          'DATABASE=Debt;'
                          'Trusted_Connection=yes;')
cursor = conn.cursor()

for ind, row in df.iterrows():
    try:
        cursor.execute('''
            insert into dbo.T_CONTRACTOR_SHERULER (NAME, SHEDULE, DATE_BEGIN, DATE_END)
            values (?,?,?,?)
            ''', row.NAME, row.SHEDULE, row.DATE_BEGIN, row.DATE_END)
    except Exception as es:
        print(row.NAME, row.SHEDULE, row.DATE_BEGIN, row.DATE_END)
conn.commit()
conn.close()

# Задание 2
``` mysql
    if object_id('dbo.T_CONTRACTOR_WORK_DAY', 'U') is not null
        drop table DEBT.dbo.T_CONTRACTOR_WORK_DAY;

    CREATE TABLE DBO.T_CONTRACTOR_WORK_DAY (
        ID INT NOT NULL  IDENTITY(1, 1),
        NAME VARCHAR(255),
        DATE_BEGIN DATETIME,
        DATE_END DATETIME,
        PRIMARY KEY (ID)
        )
```

# Задание 3

``` mysql
IF (OBJECT_ID('SelectAllCustomers') IS NOT NULL)
  DROP PROCEDURE SelectAllCustomers
GO

CREATE PROCEDURE SelectAllCustomers
	@date_begin DATE,
	@date_end DATE,
	@shedule nvarchar(50),
	@name nvarchar(100)
AS
	declare @len_date int = DATEDIFF(day, @date_begin, @date_end)
	declare @new_string nvarchar(50)
	
	set @new_string = @shedule + SUBSTRING(@shedule, 1, @len_date - len(@shedule))	
	declare @i int = 1
	declare @len_new_str int = len(@new_string)
	
	if @len_date > len(@shedule)
		while (@i <= @len_new_str)
			begin
				if substring(@new_string, @i, 1) = 'д'
					begin 
						insert into [Debt].[dbo].[T_CONTRACTOR_WORK_DAY]
						values (@name, convert(nvarchar, dateadd(hour, 8, DATEADD(day, @i-1, cast(@date_begin as nvarchar))), 120),
								convert(nvarchar, dateadd(hour, 20, DATEADD(day, @i-1, cast(@date_begin as nvarchar))), 120))
					end
				if substring(@new_string, @i, 1) = 'н'
					begin
						insert into [Debt].[dbo].[T_CONTRACTOR_WORK_DAY]
						values (@name, convert(nvarchar, dateadd(hour, 20, DATEADD(day, @i-1, cast(@date_begin as nvarchar))), 120),
							convert(nvarchar, dateadd(hour, 8, DATEADD(day, @i, cast(@date_begin as nvarchar))), 120))
					end
				if substring(@new_string, @i, 1) = 'с'
					begin
						insert into [Debt].[dbo].[T_CONTRACTOR_WORK_DAY]
						values(@name, convert(nvarchar, dateadd(hour, 8, DATEADD(day, @i-1, cast(@date_begin as nvarchar))), 120),
							convert(nvarchar, dateadd(hour, 8, DATEADD(day, @i, cast(@date_begin as nvarchar))), 120))
					end
				set @i = @i +1
			end
GO
```

# Задание 4

## 4.1

``` mysql
select name, count(date_begin)
from [T_CONTRACTOR_WORK_DAY]
group by name
```

## 4.2

``` mysql
select name, count(date_begin)
from [T_CONTRACTOR_WORK_DAY]
where date_beign < '2019-02-01'
group by name
having count(date_begin) > 10
```

## 4.3

``` mysql
select distinct name
from [T_CONTRACTOR_WORK_DAY]
where date_begin between '2019-01-14' and '2019-01-17'
```

# Задание 5

## Задагте 3 на питоне

In [ ]:
from datetime import datetime, timedelta


def get_works_days(date_begin: str, date_end: str, shedule: str, name: str):
    begin = datetime.strptime(date_begin, '%Y-%m-%d')
    end = datetime.strptime(date_end, '%Y-%m-%d')
    work_days = (end - begin).days
    
    lst_to_df = []
    
    if len(shedule) < work_days:
        shedule += shedule[:work_days - len(shedule)]
    try:
        for i in range(work_days):
            dct_to_lst = {}
            if shedule[i] == 'д':
                dct_to_lst['NAME'] = name
                dct_to_lst['DATE_BEGIN'] = begin + timedelta(days=i) + timedelta(hours=8)
                dct_to_lst['DATE_END'] = begin + timedelta(days=i) + timedelta(hours=20)
            elif shedule[i] == 'с':
                dct_to_lst['NAME'] = name
                dct_to_lst['DATE_BEGIN'] = begin + timedelta(days=i) + timedelta(hours=8)
                dct_to_lst['DATE_END'] = begin + timedelta(days=i+1) + timedelta(hours=8)
            elif shedule[i] == 'н':
                dct_to_lst['NAME'] = name
                dct_to_lst['DATE_BEGIN'] = begin + timedelta(days=i) + timedelta(hours=20)
                dct_to_lst['DATE_END'] = begin + timedelta(days=i+1) + timedelta(hours=8)
            lst_to_df.append(dct_to_lst)
    except:
        pass
    df = pd.DataFrame(lst_to_df).dropna()
    return df

df_to_load = get_works_days('2019-01-01', '2019-01-10', 'дддвсвнн', 'Поставщик 1')
df_to_load.to_sql("T_CONTRACTOR_WORK_DAY", engine ,if_exists='append', chunksize=50)

In [ ]:
# Чтобы не делать запросы к бд поработаем средствами панд

df_four_task = pd.DataFrame()
for ind, row in df.iterrows():
    df_four_task = df_four_task.append(
        get_works_days(
            str(row.DATE_BEGIN)[:10],
            str(row.DATE_END)[:10], row.SHEDULE, row.NAME), ignore_index=True)

In [ ]:
# Сделать выборку содержащую сколько рабочих дней было у каждого поставщика
df_four_task.groupby('NAME').count()

In [ ]:
# Сделать выборку поставщиков, у которыйх было больше 10 рабочих дней за январь 2019 года
df_four_task[(df_four_task.DATE_BEGIN >= '2019-01-01') & (df_four_task.DATE_BEGIN < '2019-02-01')].groupby('NAME').filter(lambda x: len(x) > 10).groupby('NAME').count()

In [ ]:
# Сделать выборку поставщиков, кто работал 14, 15 и 16 января 2019 года
df_four_task[(df_four_task.DATE_BEGIN >= '2019-01-14') & (df_four_task.DATE_BEGIN <= '2019-01-16')]['NAME'].unique().tolist()